# Explore whether the Zillow Data has the accurate substation attached

The substation attachment to the Zillow house data was completed before the  start of this project. This repository explores the accuracy of the pre-assingments. 

In [ ]:
# load necessary libraries
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import shapely # to make the lat and long of the zillow data into a gpd object
from shapely.geometry import box  # To create polygon bounding box
import glob
import folium

In [3]:
# SDGE substation polygons
sdge_substations = gpd.read_file("../../../../capstone/electrigrid/data/utilities/sandiego_grid/substation_sdge.geojson")

In [5]:
# read in the zillow geo data 
zillowmap = gpd.read_file("../../../../capstone/electrigrid/data/buildings/final_zillow.gpkg")

**Note: Zillow data takes 12 minutes to load with subpar internet**


### Clip the Zillow data using the San Diego Substation polygons

First check the CRS.

In [6]:
sdge_substations.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [7]:
zillowmap.crs

<Projected CRS: EPSG:26910>
Name: NAD83 / UTM zone 10N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: North America - between 126°W and 120°W - onshore and offshore. Canada - British Columbia; Northwest Territories; Yukon. United States (USA) - California; Oregon; Washington.
- bounds: (-126.0, 30.54, -119.99, 81.8)
Coordinate Operation:
- name: UTM zone 10N
- method: Transverse Mercator
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [10]:
# change the crs to the crs of the sd data 
sd_zillow = zillowmap.to_crs('EPSG:4326')

assert sdge_substations.crs == sd_zillow.crs

In [11]:
# clip the zillow data using the san diego
sd_zillow = gpd.clip(sd_zillow, sdge_substations)